In [61]:
import numpy as np
import yaml
import matplotlib.pyplot as plt
from CoolProp.CoolProp import PropsSI
import csv

In [62]:
with open(r'/Users/dl/Documents/GitHub/Turbopump/TCA/TCA_params.yaml') as file: ##CHANGED FOR DANIEL
		tca_params = yaml.safe_load(file)

In [63]:
# == Conversion between units ==
psi_into_pa = 6894.76 # Convert psi -> Pascal
meters_into_inches = 39.37 # Convert meters -> inches
degrees_into_rad = np.pi/180 # Convert degrees -> radians
lbm_into_kg = 0.453592 # Convert lbm -> kg
lbf_into_newton = 4.44822 # Convert lbf -> Newton
g0 = 9.81 # m/s^2

In [64]:
# == Desing Parameters (Change as needed) == 
mdot = tca_params['tca_throat_choked_flow']*lbm_into_kg #Total propellant mass flow rate [kg/s]
OF_Ratio = tca_params['oxidizer_fuel_ratio'] #Mixture ratio O/F 
rho_rp1 = 810 #RP1 Density [kg/m^3] at injector conditions
rho_lox = 1141 #LOX Density [kg/m^3] at injector conditions


Pc = tca_params['tca_chamber_pressure']*psi_into_pa #Chamber stagnation pressure [Pa]
Pin = tca_params['incoming_pressure']*psi_into_pa #injector inlet pressure [Pa]

delta_P_ox = 0.40*Pc #Pressure drop across injector [Pa]
delta_P_rp1 = 0.40*Pc #Pressure drop across injector [Pa]

Cd_ox = tca_params['discharge_coefficient_lox'] #Discharge coefficient
Cd_rp1 = tca_params['discharge_coefficient_rp1'] #Discharge coefficient
Cd_film = tca_params['discharge_coefficient_film'] #Discharge coefficient film cooling

Length_chamber = tca_params['tca_chamber_length']/meters_into_inches #Combustion chamber length [m]
CombDiam = tca_params['tca_chamber_diameter']/meters_into_inches #chamber inner diameter [m]
throat_diameter = tca_params['tca_throat_diameter']/meters_into_inches #throat diameter [m]

num_holes_rp1_inj = tca_params['injector_number_of_holes_rp1'] #number of RP1 holes in injector faceplate
num_holes_lox_inj = tca_params['injector_number_of_holes_lox'] #number of LOX holes in injector faceplate
N_film_Holes = tca_params['N_film_Holes'] #Number of film cooling holes

# == Injector Geometry Parameters ==
wall_thickness_rp1 = tca_params['injector_plate_thickness_rp1'] #[m]
wall_thickness_ox = tca_params['injector_plate_thickness_ox'] #[m]

impinge_distance = tca_params['impinge_distance']/meters_into_inches #streams will impinge at 2.2% of chamber length

#MANIFOLD DESIGN
# Allow separate manifold heights for LOX and fuel. Fall back to the
h_manifold_ox = tca_params['h_manifold_ox']

ratio_film_cooling = 0.15 #Fraction of fuel mass flow sent to film cooling vs. injection

#Friction factors and local loss coefficients
fOx = 0.01 #darcy friction factor LOX
fRP1 = 0.01 #darcy friction factor RP1
KOx = 1 #lumped local loss K LOX
KRP1 = 1 #lumped local loss K RP1
NinletsOx = 1 #number of LOX manifold inlets
NinletsRP1 = 1 #number of RP1 manifold inlets

In [65]:
sigma_rp1 = PropsSI('SURFACE_TENSION','T',298,'Q',0,'Dodecane') #Surface tension of RP-1 at injector conditions [N/m]
sigma_lox = PropsSI('SURFACE_TENSION','T',90,'Q',0,'Oxygen') #Surface tension of LOX at injector conditions [N/m]

In [66]:
mdot_kero = mdot/(1+OF_Ratio) #Fuel mass flow (RP-1)
mdot_lox = mdot*OF_Ratio/(1+OF_Ratio) #Oxidizer mass flow (LOX)

This function distribute the total propellant mass flow among a specific set of injector holes
    mdot_propellant --- the total mass flow rate of that propelant [kg/s]
    number_holes --- number of holes of this specific type 
    total_number_of_holes --- total number of holes that share the propellant flow

In [67]:
def mdot_prop(mdot_propellant, number_holes, total_number_of_holes):
    mdot_propel = mdot_propellant * number_holes/(total_number_of_holes)
    mdot_per_hole = mdot_propel/number_holes
    return mdot_propel, mdot_per_hole

mdot_rp1_inj, mdot_rp1_inj_per_hole = mdot_prop(mdot_kero, num_holes_rp1_inj, num_holes_rp1_inj)
mdot_lox_inj, mdot_lox_inj_per_hole = mdot_prop(mdot_lox, num_holes_lox_inj, num_holes_lox_inj) 

print(f"RP1 mass flow: {mdot_rp1_inj:.3f} kg/s")
print(f"LOX mass flow: {mdot_lox_inj:.3f} kg/s")

RP1 mass flow: 2.909 kg/s
LOX mass flow: 6.110 kg/s


This function computes the required injector orifice diameter for a given mass flow 
    mdot_per_prop --- mass flow rate through this specific parth [kg/s]
    rho --- propellant density at injector conditions
    delta_pressure --- pressure drop across the orifice [psi]
    number_holes --- number of identical holes sharing this mass flow

In [68]:
def diameter(mdot_per_prop, rho, delta_pressure, number_holes,Cd): 
    #following the orifice flow model, and solving for the Area
    Total_area = mdot_per_prop/(Cd*np.sqrt(2*rho*delta_pressure))
    Area_per_hole = Total_area/number_holes
    d_hole = 2*np.sqrt(Area_per_hole/np.pi) 
    return d_hole

diameter_inj_rp1 = diameter(mdot_rp1_inj, rho_rp1, delta_P_rp1, num_holes_rp1_inj,Cd_rp1)
diameter_inj_lox = diameter(mdot_lox_inj, rho_lox, delta_P_ox, num_holes_lox_inj,Cd_ox)

print(f"RP1 injector hole diameter: {diameter_inj_rp1*10**3:.3f} mm")
print(f"LOX injector hole diameter: {diameter_inj_lox*10**3:.3f} mm")


RP1 injector hole diameter: 1.107 mm
LOX injector hole diameter: 1.472 mm


This function computes the ideal jet exit velocity through an injector orifice using Bernoulli's incompressible flow relation with a pressure drop
    delta_pressure_injector --- pressure drop across the injector 
    rho --- propellant density at injector conditions [kg/m^3]

In [69]:
def velocity(delta_pressure_injector, rho):
    velocity = np.sqrt((2*delta_pressure_injector)/rho)
    return velocity

v_lox = velocity(delta_P_ox, rho_lox)
v_rp1 = velocity(delta_P_rp1, rho_rp1)

print(f"LOX velocity: {v_lox:.2f} m/s")
print(f"RP1 velocity: {v_rp1:.2f} m/s")

LOX velocity: 49.16 m/s
RP1 velocity: 58.35 m/s


In [70]:
num_quadlets = num_holes_lox_inj // 2

def find_optimal_theta_ox(theta_fuel_deg, fuel_mass_flow, ox_mass_flow, fuel_velocity, ox_velocity):
    theta_f_rad = theta_fuel_deg*degrees_into_rad
    #transverse component factor for fuel (toward center)
    sin_theta_f = np.sin(theta_f_rad)
    required_sin_theta_ox = (fuel_mass_flow * fuel_velocity * sin_theta_f) / (ox_mass_flow * ox_velocity)
    theta_ox_rad = np.arcsin(required_sin_theta_ox)
    theta_ox_deg = theta_ox_rad/degrees_into_rad
    return theta_ox_deg

def spacing_from_center(impinge_distance, theta):
    dist_impingement = impinge_distance
    r_from_center = dist_impingement*np.tan(theta*degrees_into_rad)
    return r_from_center

def new_global_OF_ratio(RP1_mass_flow_core):
    mdot_fuel_total = RP1_mass_flow_core/(1 - ratio_film_cooling)
    mdot_fuel_film = mdot_fuel_total - RP1_mass_flow_core
    OF_global_new = mdot_lox/(mdot_fuel_total)
    mdot_total_new = mdot_lox + mdot_fuel_total
    return mdot_fuel_film, mdot_fuel_total, OF_global_new, mdot_total_new



In [71]:
def compute_spacing_doublet(impingement_distance, theta_fuel_deg, theta_ox_deg):
    #compute axial impingement location
    z_imp = impingement_distance/meters_into_inches*1e3
    #converting jet angles into radians
    theta_f = theta_fuel_deg*degrees_into_rad
    theta_ox = theta_ox_deg*degrees_into_rad
    #Lateral displacement of each jet at the impingement plane:
    # x = z * tan(theta)
    d_imp_f = z_imp * (np.tan(theta_f))
    d_imp_ox = z_imp * (np.tan(theta_ox))
    #required injector-to-injector spacing for the jets to meet at z_imp
    d_fo = z_imp * (np.tan(theta_f) + np.tan(theta_ox))
    return z_imp, d_imp_f, d_imp_ox, d_fo

In [72]:
def dist(thickness, theta):
    theta_rad = theta*degrees_into_rad
    dist = thickness*(np.tan(theta_rad))
    return dist

In [73]:
def design_manifold(d_orifice, n_orifices_total, h, rho, m_dot_total, n_inlets=1):
    # compute orifices and flow handled by a single inlet/branch
    n_orifices = max(1, int(n_orifices_total / max(1, n_inlets)))
    m_dot = m_dot_total / max(1, n_inlets)

    # Total exit area for the orifices served by this inlet:
    A_exit = n_orifices * (np.pi*(d_orifice**2)/4.0)
    # required manifold cross-sectional area (4:1 rule -> manifold area = 4 * exit area)
    req_manifold_area = 4.0 * A_exit
    # geometry with rounded corners (choose corner radius r = h/4)
    r = h/4.0
    corner_area_loss = (r**2)*(4 - np.pi)
    w = (req_manifold_area)/h
    # manifold internal velocity (for this inlet/branch)
    v = m_dot/(rho*req_manifold_area) if req_manifold_area>0 else np.inf
    return A_exit, req_manifold_area, h, w, r, v

A_exit, req_manifold_area, h, w, r, v = design_manifold(diameter_inj_lox, num_holes_lox_inj, h_manifold_ox, rho_lox, mdot_lox, NinletsOx)

print(f"Req. Manifold Area: {req_manifold_area:.6f} m^2")
w11=56.795*10**-3
print(req_manifold_area/w11*1000)

Req. Manifold Area: 0.000545 m^2
9.588578929653222


In [74]:
#####MAYBE NEEDS TO CHANGE#####

def design_manifold_circular(d_orifice, n_orifices_total, rho, m_dot_total, n_inlets=1):
    # orifices and flow handled by a single inlet/branch
    n_inlets = max(1, int(n_inlets))
    n_orifices = max(1, int(n_orifices_total / n_inlets))
    m_dot = m_dot_total / n_inlets

    # total exit area for the orifices served by this inlet
    A_exit = n_orifices * (np.pi * (d_orifice**2) / 4.0)

    # 4:1 rule: manifold area = 4 * total orifice exit area
    A_manifold = 3.5 * A_exit

    # circular cross-section: A = pi D^2 / 4  -> D = sqrt(4A/pi)
    D = np.sqrt(4.0 * A_manifold / np.pi) if A_manifold > 0 else np.inf
    R = 0.5 * D if np.isfinite(D) else np.inf

    # internal velocity in this branch
    v = m_dot / (rho * A_manifold) if A_manifold > 0 else np.inf

    return A_exit, A_manifold, D, R, v

A_exit, A_manifold, Circular_mani_diameter, R, v = design_manifold_circular(
    d_orifice=tca_params['injector_hole_diameter_rp1']/1000,
    n_orifices_total=6,
    rho=rho_rp1,
    m_dot_total=0
)

print(f"RP1 manifold diameter: {Circular_mani_diameter*1000:.3f} mm")

RP1 manifold diameter: 5.005 mm


In [75]:
mdot_rp1_inj, mdot_rp1_inj_per_hole = mdot_prop(mdot_kero, num_holes_rp1_inj, num_holes_rp1_inj)
mdot_lox_inj, mdot_lox_inj_per_hole = mdot_prop(mdot_lox, num_holes_lox_inj, num_holes_lox_inj) 

print("\n=== Mass flow rate per hole ===")
print(f"RP1 injection mass flow rate per hole ({num_holes_rp1_inj}): {mdot_rp1_inj_per_hole:.5f} kg/s")
print(f"LOX injection mass flow rate per hole ({num_holes_lox_inj}): {mdot_lox_inj_per_hole:.5f} kg/s")

print(f"RP mass flow rate: {mdot_kero:.5f} kg/s")
print(f"LOX mass flow rate : {mdot_lox:.5f} kg/s")

print(f"O/F Ratio: {OF_Ratio:.3f}")


=== Mass flow rate per hole ===
RP1 injection mass flow rate per hole (80): 0.03637 kg/s
LOX injection mass flow rate per hole (80): 0.07637 kg/s
RP mass flow rate: 2.90943 kg/s
LOX mass flow rate : 6.10980 kg/s
O/F Ratio: 2.100


In [76]:
diameter_inj_rp1 = diameter(mdot_rp1_inj, rho_rp1, delta_P_rp1, num_holes_rp1_inj,Cd_rp1)
diameter_inj_lox = diameter(mdot_lox_inj, rho_lox, delta_P_ox, num_holes_lox_inj,Cd_ox)
print("\n=== Diameter of each hole ===")
print(f"The diameter of each RP1 hole into the injector is {diameter_inj_rp1*1e3:.8f} mm")
print(f"The diameter of each liquid oxygen hole into the injector is {diameter_inj_lox*1e3:.8f} mm")


=== Diameter of each hole ===
The diameter of each RP1 hole into the injector is 1.10663080 mm
The diameter of each liquid oxygen hole into the injector is 1.47201504 mm


In [77]:
theta_rp1_deg = float(input("Fuel angle (deg): "))
theta_lox_deg = find_optimal_theta_ox(theta_rp1_deg, mdot_rp1_inj, mdot_lox_inj, v_rp1, v_lox)
print(f"Optimal RP1 injection angle (deg): {theta_rp1_deg:.3f}")
print(f"Optimal LOX injection angle (deg): {theta_lox_deg:.3f}")

Optimal RP1 injection angle (deg): 32.000
Optimal LOX injection angle (deg): 17.427


In [78]:

r_fuel = spacing_from_center(impinge_distance, theta_rp1_deg)
r_ox = spacing_from_center(impinge_distance, theta_lox_deg)

print("\n=== Impingement Geometry ===")
print(f"Radius from centerline for RP1 jets at impingement: {r_fuel*1000:.5f} mm")
print(f"Radius from centerline for LOX jets at impingement: {r_ox*1000:.5f} mm")
print(f"Distance between RP1 jets: {r_fuel*1000*2:.5f} mm")


=== Impingement Geometry ===
Radius from centerline for RP1 jets at impingement: 3.86318 mm
Radius from centerline for LOX jets at impingement: 1.94068 mm
Distance between RP1 jets: 7.72635 mm


In [79]:
def dist(thickness, theta):
    theta_rad = theta*degrees_into_rad
    dist = thickness*(np.tan(theta_rad))
    return dist



print(f"RP1, distance, for angle: {dist(wall_thickness_rp1, theta_rp1_deg)*1000:.5f} mm")
print(f"LOX, distance, for angle: {dist(wall_thickness_ox, theta_lox_deg)*1000:.5f} mm") 

RP1, distance, for angle: 3.74922 mm
LOX, distance, for angle: 2.82514 mm


In [80]:
We_lox = (rho_lox * (v_lox*np.sin(new_ox_angle*degrees_into_rad))**2 * diameter_inj_lox) / sigma_lox
We_rp1 = (rho_rp1 * (v_rp1*np.sin(theta_rp1_deg*degrees_into_rad))**2 * diameter_inj_rp1) / sigma_rp1
print("\n=== Weber Number Check ===")
print(f"Weber Number for LOX jets: {We_lox:.2f}")
print(f"Weber Number for RP1 jets: {We_rp1:.2f}")


=== Weber Number Check ===
Weber Number for LOX jets: 32300.15
Weber Number for RP1 jets: 34358.10
